In [7]:
%matplotlib qt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
import mne
import os
import numpy as np
from tqdm.contrib.itertools import product
from tqdm.notebook import tqdm
import pickle
from scipy.stats import ttest_rel, pearsonr
import scipy
from matplotlib import colors as mcolors
from mne.datasets import fetch_fsaverage
from mne.stats import permutation_cluster_1samp_test

Load the Dataframe / define responders

In [2]:
## load power in labels and add whole brain activation
dfs_list = []
folder_name = Path.cwd().parent / "data" / "dataframes" / "powers"
for file in tqdm(sorted(os.listdir(folder_name))):
    if file.endswith(".csv"):
        fname = folder_name / file
        df = pd.read_csv(fname, index_col=[0])
        
        ## add whole brain power
        df1s_list = []
        for frange in ["delta", "theta", "alpha", "beta", "gamma"]:
            for hemi in ["lh", "rh"]:
                df1 = df[df["frequency_band"] == frange]
                df1 = df1[df1['brain_label'].str.contains(f'-{hemi}', case=False, na=False)]
                power_hemi = df1["power"].sum()
                keys = list(df.columns)
                values = [df1[key].unique()[0] for key in keys[:-2]]
                values.append([f"whole-{hemi}"])
                values.append([power_hemi])
                new_row = dict(zip(keys, values))
                df_new_row = pd.DataFrame(new_row)
                df1 = pd.concat([df1, df_new_row])
                df1s_list.append(df1)
        df_updated = pd.concat(df1s_list)
        dfs_list.append(df_updated)
        
dfs = pd.concat(dfs_list)
df = dfs.reset_index().drop(columns=["index", "Unnamed: 0"])

## define responders and non responders
# subjects_to_drop_dict = {}
# prots = ["1 Hz", "10 Hz", "20 Hz"]
# hemis = ["left", "right"]
# all_combs = product(prots, hemis)
# for protocol, hemi in all_combs:
#     fname = "/Users/payamsadeghishabestari/codes/regTMS/data/behavioral_data/TL_data.xlsx"
#     df_tl = pd.read_excel(fname)
#     df_tl["protcol"] = df_tl["protcol"].replace("1Hz", "1 Hz")
#     df_tl_sub = df_tl.query(f'protcol == "{protocol}" & hemisphere == "{hemi}"')
#     df_tl_sub['pre_avg'] = df_tl.filter(like='pre').mean(axis=1)
#     df_tl_sub['post_avg'] = df_tl.filter(like='post').mean(axis=1)
#     df_tl_sub["diff_TL"] = df_tl_sub['pre_avg'] - df_tl_sub['post_avg'] # post_TL_0
#     df_tl_sub = df_tl_sub.rename(columns={'ID': 'subject_ID'})
#     df_tl_diff = df_tl_sub[["subject_ID", "diff_TL"]]
#     df_tl_diff = df_tl_diff.query('diff_TL <= 0')
#     subjects_to_drop_dict[f"{hemi}_{protocol}"] = list(df_tl_diff["subject_ID"].values)

## load the dictionaries
folder_name = Path.cwd().parent / "data" / "dataframes" / "results_new"

with open(folder_name / "whole_bl.pkl", 'rb') as pickle_file:
    dict_1 = pickle.load(pickle_file)
with open(folder_name / "whole_compared_to_sham_bl.pkl", 'rb') as pickle_file:
    dict_2 = pickle.load(pickle_file)

## loop over brain labels to find commons
comm_dict = {}
for key in dict_1:
    vals_1 = dict_1[key]
    vals_2 = dict_2[key]
    common_elements = set(vals_1) & set(vals_2)
    if len(common_elements):
        comm_dict[key] = list(common_elements)

  0%|          | 0/353 [00:00<?, ?it/s]

Check that pre vs post and varum vs sham both are significant

In [ ]:
brain_labels = np.array(mne.read_labels_from_annot(subject="fsaverage", parc="aparc", verbose=False))[:-1]
lb_names = [lb.name for lb in brain_labels]
protocols = ["1 Hz", "10 Hz", "20 Hz"]
franges = ["delta", "theta", "alpha", "beta", "gamma"]
hemis = ["left", "right"]
combs = product(hemis, protocols, franges, lb_names)
my_dict = {"hemi": [], "protocol": [], "frange": [], "lb_name": []}

for hemi, protocol, frange, lb_name in combs:
    df1 = df.query(f'hemisphere == "{hemi}" & protocol == "{protocol}" & frequency_band == "{frange}" & brain_label == "{lb_name}"')
    df_pivot = df1.pivot(index='subject_ID', columns='run', values='power')
    df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
    df_pivot = df_pivot.reset_index()
    t_val, p_val_1 = ttest_rel(df_pivot["pre"], df_pivot["post"])


    df_sham = df.query(f'hemisphere == "{hemi}" & protocol == "0.1 Hz" & frequency_band == "{frange}" & brain_label == "{lb_name}"')
    df_pivot_sh = df_sham.pivot(index='subject_ID', columns='run', values='power')
    df_pivot_sh['power_difference'] = df_pivot_sh['post'] - df_pivot_sh['pre']
    df_pivot_sh = df_pivot_sh.reset_index()
    t_val, p_val_2 = ttest_rel(df_pivot["power_difference"], df_pivot_sh["power_difference"])

    if p_val_1 < 0.05 and p_val_2 < 0.05:
        my_dict["hemi"].append(hemi)
        my_dict["protocol"].append(protocol)
        my_dict["frange"].append(frange)
        my_dict["lb_name"].append(lb_name)
        # my_dict["t_val"].append(t_val)
        # my_dict["p_val"].append(p_val)

df_double_sig = pd.DataFrame(my_dict)


Picture 1.a (sensors with brain)

In [ ]:
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = fs_dir.parent
subject = "fsaverage"
src = fs_dir / "bem" / "fsaverage-ico-5-src.fif"
bem = fs_dir / "bem" / "fsaverage-5120-5120-5120-bem-sol.fif"

input_fname = "/Users/payamsadeghishabestari/codes/regTMS/data/EEG_data/2bbyx_1_L_0.1Hz_post.set"
raw = mne.io.read_raw_eeglab(input_fname=input_fname,
                            preload=True, verbose=False)
montage = mne.channels.make_standard_montage("standard_1020")
raw.set_montage(montage=montage, match_case=False, on_missing="raise")
info = raw.info

sensor_colors_dict = {}
sensor_colors = ['#1f77b4'] * len(info["ch_names"])
for ch in ["CP5", "CP6"]:
    ch_idx = info["ch_names"].index(ch)
    sensor_colors[ch_idx] = "red"
sensor_colors_dict["eeg"] = sensor_colors
surfaces = dict(brain=0.8, seghead=0.2)
fig = mne.viz.create_3d_figure(size=(600, 600), bgcolor=(255, 255, 255))
mne.viz.plot_alignment(
    info = raw.info,
    trans="fsaverage",
    subject="fsaverage",
    surfaces=surfaces,
    eeg=dict(original=0.4, projected=0.9),
    coord_frame="auto",
    show_axes=False,
    mri_fiducials=False,
    dig=False,
    sensor_colors=sensor_colors_dict,
    fig=fig)

mne.viz.set_3d_view(figure=fig, azimuth=180, elevation=90, distance=0.6) # azimuth=180
screenshot = fig.plotter.screenshot()
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(screenshot)
ax.set_axis_off()
fig.tight_layout()
fig.savefig(Path.cwd().parent / "figures" / "eeg_sensors_2.pdf")

Picture 1.b (significant topomaps pre/post)

In [ ]:
input_fname = "/Users/payamsadeghishabestari/codes/regTMS/data/EEG_data/2bbyx_1_L_0.1Hz_post.set"
raw = mne.io.read_raw_eeglab(input_fname=input_fname,
                            preload=True, verbose=False)
montage = mne.channels.make_standard_montage("easycap-M1")
raw.set_montage(montage=montage, match_case=False, on_missing="raise")
info = raw.info

folder_name = Path.cwd().parent / "data" / "dataframes" / "sensor"
subject_ids = []
for file in sorted(os.listdir(folder_name)):
    if file.endswith(".pkl"):
        subject_ids.append(file[:5])
subject_ids = list(set(subject_ids))

mask_params = dict(marker='o', markerfacecolor='w', markeredgecolor='k',
                    linewidth=0, markersize=6)
p_thr = 0.05
protocols = ["1Hz", "10Hz", "20Hz"]
franges = ["delta", "theta", "alpha", "beta", "gamma"]
hemis = ["left", "right"]
combs = product(hemis, protocols, franges)

adjacency, ch_names = mne.channels.find_ch_adjacency(raw.info, ch_type="eeg")
pval = 0.05  # arbitrary
df = 22 - 1  
thresh = scipy.stats.t.ppf(1 - pval / 2, df)

for hemi, protocol, frange in combs:
    x1, x2 = [], []
    for subject_id in subject_ids:
        fname_pre = folder_name / f"{subject_id}_{protocol}_{hemi}_pre.pkl"
        fname_post = folder_name / f"{subject_id}_{protocol}_{hemi}_post.pkl"

        with open(fname_pre, "rb") as fpre:
            dict_pre = pickle.load(fpre)
        with open(fname_post, "rb") as fpost:
            dict_post = pickle.load(fpost)

        x1.append(dict_pre[frange].mean(axis=0))
        x2.append(dict_post[frange].mean(axis=0))

    X = np.array(x2) - np.array(x1)
    t_obs, clus, clus_p, _ = permutation_cluster_1samp_test(X=X, threshold=thresh,
                                                            adjacency=adjacency,
                                                            n_permutations=1000,
                                                            verbose=False)
    if len(clus) > 0:
        print(f"{protocol}_{hemi}_{frange}_{clus}")
        mask = np.zeros(len(t_obs))
        for cl in clus:
            mask[cl[0]] = 1
        mask = np.bool_(mask)
        evoked = mne.EvokedArray(t_obs[:, np.newaxis], info, tmin=0.0)
        fig, ax = plt.subplots(1, 1, figsize=(5, 5), layout="constrained")
        evoked.plot_topomap(ch_type="eeg", times=[0], scalings=1, colorbar=False,
            time_format=None, cmap="RdBu_r", vlim=(0, 2), units="t_val",
            cbar_fmt="%0.1f", mask=mask[:,np.newaxis], mask_params=mask_params,
            size=3, show_names=False, time_unit="s", axes=ax)
        ax.set_title(f"{protocol}_{hemi}_{frange}")
        fig.savefig(Path.cwd().parent / "figures" / "to_stefan" / "topomaps" / "pre_vs_post" / f"{protocol}_{hemi}_{frange}.pdf")

Picture 2.a (source space)

In [ ]:
brain_labels = np.array(mne.read_labels_from_annot(subject="fsaverage", parc="aparc", verbose=False))
lb_names = [lb.name for lb in brain_labels]
brain_kwargs = dict(background="white", surf="pial_semi_inflated", cortex=["#b8b4ac", "#b8b4ac"])
flattened_list = [item for sublist in comm_dict.values() for item in sublist]
all_regions = list(set(flattened_list))

all_regions = ["insula-rh", "lateraloccipital-lh", "entorhinal-rh",
                "parsorbitalis-rh", "entorhinal-lh", "posteriorcingulate-rh",
                "superiorfrontal-rh", "rostralmiddlefrontal-rh", "middletemporal-rh",
                "medialorbitofrontal-rh", "caudalmiddlefrontal-rh", "supramarginal-rh",
                "cuneus-lh", "fusiform-rh", "precuneus-lh", "rostralanteriorcingulate-lh",
                "pericalcarine-lh", "temporalpole-lh", "lateralorbitofrontal-lh",
                "frontalpole-rh", "postcentral-rh", "rostralanteriorcingulate-rh",
                "frontalpole-lh", "medialorbitofrontal-lh", "parsopercularis-rh",
                "caudalanteriorcingulate-rh", "parstriangularis-rh", "inferiortemporal-rh",
                "superiortemporal-rh", "lateralorbitofrontal-rh", "lingual-lh", "temporalpole-rh"]

region_idxs = [all_regions.index(rg) for rg in \
                ["rostralanteriorcingulate-lh", "precuneus-lh", "inferiortemporal-rh", "supramarginal-rh"]]

colors_1 = sns.color_palette(palette="tab20", n_colors=20)
colors_2 = sns.color_palette(palette="Set2", n_colors=8)
colors_3 = sns.color_palette(palette="dark", n_colors=4)

extended_colors = colors_1 + colors_2 + colors_3

alpha = 0.6
franges = ["delta", "theta", "alpha", "gamma"]

new_cls = []
new_cls.append(sns.color_palette("rocket", as_cmap=False, n_colors=10)[1])
new_cls.append(sns.color_palette("rocket", as_cmap=False, n_colors=10)[3])
new_cls.append(sns.color_palette("mako", as_cmap=False, n_colors=10)[4])
new_cls.append(sns.color_palette("mako", as_cmap=False, n_colors=10)[7])

for idx, new_cl in zip(region_idxs, new_cls):
    extended_colors[idx] = new_cl

colors_dict = dict(zip(all_regions, extended_colors))
for key, value in comm_dict.items():
    for frange in franges:
        if frange in key:
            idxs = np.array([lb_names.index(lb) for lb in value])
            lbs = list(brain_labels[idxs])
            brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="lh", views="lateral", **brain_kwargs)
            for lb, lb_name in zip(lbs, value):
                color = colors_dict[lb_name]
                if lb.hemi == "lh":
                    brain.add_label(lb, hemi="lh", color=color, borders=False, alpha=alpha)

            brain_scr_1 = brain.screenshot()
            brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="rh", views="lateral", **brain_kwargs)
            for lb, lb_name in zip(lbs, value):
                color = colors_dict[lb_name]
                if lb.hemi == "rh":
                    brain.add_label(lb, hemi="rh", color=color, borders=False, alpha=alpha)

            brain_scr_2 = brain.screenshot()
            
            brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="lh", views="medial", **brain_kwargs)
            for lb, lb_name in zip(lbs, value):
                color = colors_dict[lb_name]
                if lb.hemi == "lh":
                    brain.add_label(lb, hemi="lh", color=color, borders=False, alpha=alpha)

            brain_scr_3 = brain.screenshot()
            brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="rh", views="medial", **brain_kwargs)
            for lb, lb_name in zip(lbs, value):
                color = colors_dict[lb_name]
                if lb.hemi == "rh":
                    brain.add_label(lb, hemi="rh", color=color, borders=False, alpha=alpha)

            brain_scr_4 = brain.screenshot()
            fig, axes = plt.subplots(2, 2, figsize=(9, 7))
            fig.subplots_adjust(hspace=0.1)
            for ax, brain in zip([axes[0][0], axes[0][1], axes[1][0], axes[1][1]], [brain_scr_1, brain_scr_2, brain_scr_3, brain_scr_4]):
                nonwhite_pix = (brain != 255).any(-1)
                nonwhite_row = nonwhite_pix.any(1)
                nonwhite_col = nonwhite_pix.any(0)
                cropped_screenshot = brain[nonwhite_row][:, nonwhite_col]
                ax.imshow(cropped_screenshot)
                ax.axis("off")
            fig.savefig(Path.cwd().parent / "figures" / f"brain_{key}.pdf")

palette_with_alpha = [mcolors.to_rgba(color, alpha=alpha) for color in extended_colors]
fig, ax = plt.subplots(1, 1, figsize=(16, 8))
ax.barh(range(len(palette_with_alpha)), [1]*len(palette_with_alpha), color=palette_with_alpha, tick_label=all_regions)
#ax.yaxis.tick_right()
ax.set_yticklabels(all_regions, fontstyle='italic', fontsize=11, ha='left')
ax.yaxis.set_tick_params(pad=-860)
for spine in ax.spines.values():
    spine.set_visible(False)
fig.savefig(Path.cwd().parent / "figures" / "colors_legend_brain.pdf")

Picture 2.b (Correlation with TL)

In [ ]:
fname = "/Users/payamsadeghishabestari/codes/regTMS/data/behavioral_data/TL_data_new.xlsx" # previously was TL_data
df_tl = pd.read_excel(fname)
df_tl["protcol"] = df_tl["protcol"].replace("1Hz", "1 Hz")
df_tl_sub = df_tl.query(f'protcol == "20 Hz" & hemisphere == "right"')
df_tl_sub['pre_avg'] = df_tl.filter(like='pre').mean(axis=1)
df_tl_sub['post_avg'] = df_tl.filter(like='post').mean(axis=1)
df_tl_sub["diff_TL"] = df_tl_sub['pre_avg'] - df_tl_sub['post_avg']
df_tl_sub = df_tl_sub.rename(columns={'ID': 'subject_ID'})
df_tl_diff = df_tl_sub[["subject_ID", "diff_TL"]]
# df_tl_diff = df_tl_diff.query('diff_TL > 0')

####

protocols = ["10 Hz", "20 Hz"]
franges = ["delta", "theta", "alpha", "beta", "gamma"]
hemis = ["left", "right"]
combs = product(hemis, protocols, franges)
corr_dict = {"protocol": [], "hemi": [], "frange": [], "brain_label": [],
            "R_value": [], "corr": [], "p_value": []}

for hemi, protocol, frange in combs:
    for key, value in comm_dict.items():
        if hemi in key and protocol in key and frange in key:
            for bl in value:
                df_sub = df.query(f'hemisphere == "{hemi}" & protocol == "{protocol}" & frequency_band == "{frange}" & brain_label == "{bl}"')
                df_pivot = df_sub.pivot(index='subject_ID', columns='run', values='power')
                df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
                merged_df = pd.merge(df_tl_diff, df_pivot, on="subject_ID")
                merged_df["frange"] = len(merged_df) * [frange]
                merged_df["brain_label"] = len(merged_df) * [bl]
                merged_df.drop(columns=["pre", "post"], inplace=True)
                corr = merged_df["diff_TL"].corr(merged_df["power_difference"])
                r_value, pea_value = pearsonr(merged_df["diff_TL"], merged_df['power_difference'])

                corr_dict["protocol"].append(protocol)
                corr_dict["hemi"].append(hemi)
                corr_dict["frange"].append(frange)
                corr_dict["brain_label"].append(bl)
                corr_dict["R_value"].append(r_value)
                corr_dict["corr"].append(corr)
                corr_dict["p_value"].append(pea_value)
                
df_corr = pd.DataFrame(corr_dict)
df_sorted = df_corr.sort_values(by='p_value')
# df_sorted.to_csv(Path.cwd().parent / "figures" / "corrs_whole.csv")

Picture 3.a (Significant topomaps wrt sham)

In [ ]:
input_fname = "/Users/payamsadeghishabestari/codes/regTMS/data/EEG_data/2bbyx_1_L_0.1Hz_post.set"
raw = mne.io.read_raw_eeglab(input_fname=input_fname,
                            preload=True, verbose=False)
montage = mne.channels.make_standard_montage("easycap-M1")
raw.set_montage(montage=montage, match_case=False, on_missing="raise")
info = raw.info

folder_name = Path.cwd().parent / "data" / "dataframes" / "sensor"
subject_ids = []
for file in sorted(os.listdir(folder_name)):
    if file.endswith(".pkl"):
        subject_ids.append(file[:5])
subject_ids = list(set(subject_ids))

mask_params = dict(marker='o', markerfacecolor='w', markeredgecolor='k',
                    linewidth=0, markersize=6)
p_thr = 0.05
protocols = ["1Hz", "10Hz", "20Hz"]
franges = ["delta", "theta", "alpha", "beta", "gamma"]
hemis = ["left", "right"]
combs = product(hemis, protocols, franges)

adjacency, ch_names = mne.channels.find_ch_adjacency(raw.info, ch_type="eeg")
pval = 0.05  # arbitrary
df = 22 - 1  
thresh = scipy.stats.t.ppf(1 - pval / 2, df)

for hemi, protocol, frange in combs:
    
    x1, x2 = [], []
    y1, y2 = [], []
    for subject_id in subject_ids:
        fname_pre = folder_name / f"{subject_id}_{protocol}_{hemi}_pre.pkl"
        fname_post = folder_name / f"{subject_id}_{protocol}_{hemi}_post.pkl"

        fname_pre_sham = folder_name / f"{subject_id}_0.1Hz_{hemi}_pre.pkl"
        fname_post_sham = folder_name / f"{subject_id}_0.1Hz_{hemi}_post.pkl"

        with open(fname_pre, "rb") as fpre:
            dict_pre = pickle.load(fpre)
        with open(fname_post, "rb") as fpost:
            dict_post = pickle.load(fpost)

        with open(fname_pre_sham, "rb") as fpre_sh:
            dict_pre_sham = pickle.load(fpre_sh)
        with open(fname_post_sham, "rb") as fpost_sh:
            dict_post_sham = pickle.load(fpost_sh)

        x1.append(dict_pre[frange].mean(axis=0))
        x2.append(dict_post[frange].mean(axis=0))

        y1.append(dict_pre_sham[frange].mean(axis=0))
        y2.append(dict_post_sham[frange].mean(axis=0))

    X = np.array(x2) - np.array(x1)
    Y = np.array(y2) - np.array(y1)
    t_obs, clus, clus_p, _ = permutation_cluster_1samp_test(X=X-Y, threshold=thresh,
                                                            adjacency=adjacency,
                                                            n_permutations=1000,
                                                            verbose=False)
    if len(clus) > 0:
        print(f"{protocol}_{hemi}_{frange}_{clus}")
        mask = np.zeros(len(t_obs))
        for cl in clus:
            mask[cl[0]] = 1
        mask = np.bool_(mask)
        evoked = mne.EvokedArray(t_obs[:, np.newaxis], info, tmin=0.0)
        fig, ax = plt.subplots(1, 1, figsize=(5, 5), layout="constrained")
        evoked.plot_topomap(ch_type="eeg", times=[0], scalings=1, colorbar=False,
            time_format=None, cmap="RdBu_r", vlim=(0, 2), units="t_val",
            cbar_fmt="%0.1f", mask=mask[:,np.newaxis], mask_params=mask_params,
            size=3, show_names=False, time_unit="s", axes=ax)
        ax.set_title(f"{protocol}_{hemi}_{frange}")
        fig.savefig(Path.cwd().parent / "figures" / "to_stefan" / "topomaps" / "prot_vs_sham" / f"{protocol}_{hemi}_{frange}.pdf")

Picture 3.b (anatomical label)

In [ ]:
## anatomical ROI
roi = ['transversetemporal', 'bankssts', 'inferiorparietal',
        'superiorparietal', 'superiortemporal', 'supramarginal',
        'middletemporal']
roi_lh = [bl + "-lh" for bl in roi]
roi_rh = [bl + "-rh" for bl in roi]
roi = roi_lh + roi_rh

df_sub = df.query(f'brain_label == @roi')
df_sub_1 = df_sub[(df_sub['hemisphere'] == 'right') & (df_sub['brain_label'].str.endswith('rh'))]
df_sub_2 = df_sub[(df_sub['hemisphere'] == 'left') & (df_sub['brain_label'].str.endswith('lh'))]
df_sub = pd.concat([df_sub_1, df_sub_2])
df_sub = df_sub.groupby(["subject_ID", "protocol", "hemisphere", "run", "frequency_band"])['power'].sum().reset_index()
df_pivot = df_sub.pivot(index=["subject_ID", "protocol", "hemisphere", "frequency_band"], columns='run', values='power')
df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
df_pivot.drop(columns=["post", "pre"], inplace=True)
df_pivot = df_pivot.reset_index()

row_order = ["delta", "theta", "alpha", "beta", "gamma"]
col_order = ["left", "right"]
size = 3
pal = sns.cubehelix_palette(7, rot=-.25, light=.7, as_cmap=False)[3]
g = sns.FacetGrid(data=df_pivot,
                row="frequency_band",
                col="hemisphere",
                height=2,
                aspect=1.8,
                palette=None,
                row_order=row_order,
                col_order=col_order,
                sharex=False,
                sharey=False)
g.map_dataframe(sns.boxplot, x="protocol", y="power_difference", fill=False, color=pal)
g.map_dataframe(sns.stripplot, x="protocol", y="power_difference", color=pal, size=size)

for (row_val, col_val), ax in g.axes_dict.items():
        if row_val == "delta": ax.set_ylim([-100, 210])
        if row_val == "theta": ax.set_ylim([-60, 60])
        if row_val == "alpha": ax.set_ylim([-60, 60])
        if row_val == "beta": ax.set_ylim([-60, 60])
        if row_val == "gamma": ax.set_ylim([-70, 100])

        ax.set_title(f"{row_val} & {col_val}", fontsize=8)
        ax.set_ylabel(f"power_diff", fontsize=8)
        ax.axvspan(-0.5, 0.5, color="grey", alpha=0.1)

        df_pivot_sub = df_pivot.query(f'hemisphere == "{col_val}" & protocol == "0.1 Hz" & frequency_band == "{row_val}"')
        ax.hlines(y=df_pivot_sub["power_difference"].median(), xmin=-0.8, xmax=4, linestyles="--", colors="grey")

g.tight_layout()
#g.savefig(Path.cwd().parent / "figures" / "to_stefan" / f"ROI.pdf")

protocols = ["1 Hz", "10 Hz", "20 Hz"]
hemis = ["left", "right"]
franges = ["delta", "theta", "alpha", "beta", "gamma"]
combs = product(hemis, protocols, franges)
for hemi, protocol, frange in combs:
        df_sub_sub = df_sub.query(f'protocol == "{protocol}" & hemisphere == "{hemi}" & frequency_band == "{frange}"')
        group_pre = df_sub_sub[df_sub_sub["run"] == "pre"]["power"].values
        group_post = df_sub_sub[df_sub_sub["run"] == "post"]["power"].values

        assert group_pre.shape == group_post.shape, "sth wrong"
        t_val, p_val = ttest_rel(group_pre, group_post)

        if p_val < 0.05:
                print(f"{frange}_{hemi}_{protocol}_{round(p_val, 3)}")

TO DO: compare the power change (post-pre) among all
protocols (only for ROI) per frequency band

In [ ]:
## anatomical ROI
roi = ['transversetemporal', 'bankssts', 'inferiorparietal',
        'superiorparietal', 'superiortemporal', 'supramarginal',
        'middletemporal']
roi_lh = [bl + "-lh" for bl in roi]
roi_rh = [bl + "-rh" for bl in roi]
roi = roi_lh + roi_rh

df_sub = df.query(f'brain_label == @roi')
df_sub = df_sub.groupby(["subject_ID", "protocol", "hemisphere", "run", "frequency_band"])['power'].sum().reset_index()
df_pivot = df_sub.pivot(index=["subject_ID", "protocol", "hemisphere", "frequency_band"], columns='run', values='power')
df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
df_pivot.drop(columns=["post", "pre"], inplace=True)
df_pivot = df_pivot.reset_index()

protocols = ["0.1 Hz", "1 Hz", "10 Hz", "20 Hz"]
hemis = ["left", "right"]
franges = ["delta", "theta", "alpha", "beta", "gamma"]
combs = product(hemis, protocols, franges)
for hemi, protocol, frange in combs:
        df_sub_sub = df_sub.query(f'protocol == "{protocol}" & hemisphere == "{hemi}" & frequency_band == "{frange}"')
        group_pre = df_sub_sub[df_sub_sub["run"] == "pre"]["power"].values
        group_post = df_sub_sub[df_sub_sub["run"] == "post"]["power"].values

        assert group_pre.shape == group_post.shape, "sth wrong"
        t_val, p_val = ttest_rel(group_pre, group_post)

        if p_val < 0.05:
                print(f"{frange}_{hemi}_{protocol}_{round(p_val, 3)}")


my_dict = {"prot_1": [], "prot_2": [], "frange": [], "avg_power_difference_1": [],
            "avg_power_difference_2": [], "t_val": [], "p_val": [], "significant": []}
for freq in franges:
    for prot_1 in protocols:
        for prot_2 in protocols:
            for hemi_1 in hemis:
                for hemi_2 in hemis:
                    if not (prot_1 == prot_2 and hemi_1 == hemi_2):
                        df_pivot_sub_1 = df_pivot.query(f'protocol == "{prot_1}" & hemisphere == "{hemi_1}" & frequency_band == "{freq}"')
                        df_pivot_sub_2 = df_pivot.query(f'protocol == "{prot_2}" & hemisphere == "{hemi_2}" & frequency_band == "{freq}"')
                        group_1 = df_pivot_sub_1["power_difference"].values
                        group_2 = df_pivot_sub_2["power_difference"].values

                        assert group_pre.shape == group_post.shape, "sth wrong"
                        t_val, p_val = ttest_rel(group_1, group_2)


                        my_dict["prot_1"].append(prot_1 + " " + hemi_1)
                        my_dict["prot_2"].append(prot_2 + " " + hemi_2)
                        my_dict["frange"].append(freq)
                        my_dict["avg_power_difference_1"].append(group_1.mean())
                        my_dict["avg_power_difference_2"].append(group_2.mean())
                        my_dict["t_val"].append(round(t_val, 4))
                        my_dict["p_val"].append(round(p_val, 4))
                        if p_val < 0.05: 
                            my_dict["significant"].append("True")
                        else:
                            my_dict["significant"].append("False")

df_roi_sig = pd.DataFrame(my_dict)

In [ ]:
df_roi_sig.to_csv("/Users/payamsadeghishabestari/Desktop/df_roi_sig.csv")

TO DO: analyze power changes from pre to post for the
responder group (same as for the whole group, but we
have only one „protocol“ the individual best ones)

In [29]:
fname = "/Users/payamsadeghishabestari/codes/regTMS/data/behavioral_data/TL_data_new.xlsx"  # changed to new
df_tl = pd.read_excel(fname)
df_tl["protcol"] = df_tl["protcol"].replace("1Hz", "1 Hz")
df_tl.rename(columns={"protcol": "protocol", "ID": "subject_ID"}, inplace=True)
prots = ["1 Hz", "10 Hz", "20 Hz"]
hemis = ["left", "right"]

best_protocol_dict = {"subject_ID": [], "hemisphere": [], "protocol": [], "TL_diff": []} 
for subject in df["subject_ID"].unique():
    prot_dict = {}
    for prot in prots:
        for hemi in hemis:
            df_tl_sub = df_tl.query(f'subject_ID == "{subject}" & protocol == "{prot}" & hemisphere == "{hemi}"')
            df_tl_sub['pre_avg'] = df_tl.filter(like='pre').mean(axis=1)
            df_tl_sub['post_avg'] = df_tl.filter(like='post').mean(axis=1)
            df_tl_sub["diff_TL"] = df_tl_sub['pre_avg'] - df_tl_sub['post_avg']
            prot_dict[f"{hemi}_{prot}"] = df_tl_sub["diff_TL"].values[0]
    best_protocol = max(prot_dict, key=prot_dict.get)
    worst_protocol = min(prot_dict, key=prot_dict.get)
    print(f"{subject}_{worst_protocol}_{prot_dict[worst_protocol]}")
    
    best_protocol_dict["subject_ID"].append(subject)
    best_protocol_dict["hemisphere"].append(best_protocol.partition("_")[0])
    best_protocol_dict["protocol"].append(best_protocol.partition("_")[2])
    best_protocol_dict["TL_diff"].append(prot_dict[best_protocol])

    best_protocol_dict["subject_ID"].append(subject)
    best_protocol_dict["hemisphere"].append(best_protocol.partition("_")[0])
    best_protocol_dict["protocol"].append("0.1 Hz")
    best_protocol_dict["TL_diff"].append(0)

df_best_prot = pd.DataFrame(best_protocol_dict)
df_best_prot = df_best_prot.query('TL_diff > 0')
df_only_responders = pd.merge(df_best_prot, df, on=["subject_ID", "hemisphere", "protocol"])

/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_1321/731265223.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tl_sub['pre_avg'] = df_tl.filter(like='pre').mean(axis=1)
/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_1321/731265223.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tl_sub['post_avg'] = df_tl.filter(like='post').mean(axis=1)
/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_1321/731265223.py:16: SettingWithCopyWarning: 
A value is trying to be set 

1045i_left_10 Hz_0.0
20cz9_right_1 Hz_-8.57142857142857
2bbyx_right_1 Hz_0.0
2fjeu_left_1 Hz_0.0
3dx2e_left_20 Hz_-1.7142857142857082
6eayz_right_1 Hz_0.0
6mb60_left_10 Hz_-1.5714285714285694
6sjul_left_1 Hz_0.0
6wms4_left_1 Hz_1.4285714285714306
8kmc7_right_10 Hz_-0.7142857142857082
cmh15_left_1 Hz_0.0
dws0m_left_1 Hz_0.0
gigcm_right_1 Hz_0.0
hqj3h_left_1 Hz_0.0
kicyy_left_10 Hz_0.0
musky_right_1 Hz_2.142857142857139
nbauc_left_1 Hz_0.0
q3xk4_left_1 Hz_4.285714285714292
uvxfg_left_1 Hz_0.0
w1p3k_right_1 Hz_-4.285714285714292
z45am_left_1 Hz_-2.857142857142861
zy37y_left_10 Hz_5.0


/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_1321/731265223.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tl_sub['pre_avg'] = df_tl.filter(like='pre').mean(axis=1)
/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_1321/731265223.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tl_sub['post_avg'] = df_tl.filter(like='post').mean(axis=1)
/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_1321/731265223.py:16: SettingWithCopyWarning: 
A value is trying to be set 

In [37]:
df_tl

,NO.,subject_ID,hemisphere,protocol,pre_TL_0,pre_TL_30,pre_TL_60,pre_TL_90,pre_TL_120,pre_TL_150,pre_TL_180,post_TL_0,post_TL_30,post_TL_60,post_TL_90,post_TL_120,post_TL_150,post_TL_180
0,X1,1045i,right,0.1 Hz,100,80,80,70,70,70,70,70,70,60,60,70,70,70
1,X1,1045i,right,10 Hz,70,90,90,90,90,90,90,80,80,80,80,80,80,80
2,X1,1045i,right,1 Hz,80,90,90,90,90,90,90,80,70,70,70,70,70,70
3,X1,1045i,right,20 Hz,70,70,70,70,70,70,70,70,70,70,70,70,70,70
4,X1,1045i,left,0.1 Hz,90,90,90,90,90,90,90,90,90,90,90,90,90,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,X22,zy37y,right,20 Hz,90,90,90,90,90,90,90,85,80,80,80,80,80,80
172,X22,zy37y,left,0.1 Hz,100,90,90,90,90,90,90,90,90,90,90,90,80,80
173,X22,zy37y,left,20 Hz,90,90,90,90,90,90,90,80,80,80,75,80,80,80
174,X22,zy37y,left,1 Hz,100,100,100,100,90,90,90,90,90,90,90,90,90,90


In [30]:
## sig brain regions in responders
brain_labels = mne.read_labels_from_annot(subject="fsaverage", parc="aparc", verbose=False)[:-1]
lb_names = [lb.name for lb in brain_labels]
franges = ["delta", "theta", "alpha", "beta", "gamma"]
combs = product(franges, lb_names)
dfs_list = []
for freq, bl in combs:
    df1 = df_only_responders.query(f'frequency_band == "{freq}" & brain_label == "{bl}"')
    group_pre = df1[df1["run"] == "pre"]["power"].values
    group_post = df1[df1["run"] == "post"]["power"].values
    t_val, p_val = ttest_rel(group_pre, group_post)
    if p_val < 0.05:
        dfs_list.append(df1)

df_final = pd.concat(dfs_list)

  0%|          | 0/340 [00:00<?, ?it/s]

In [31]:
df_final

,subject_ID,hemisphere,protocol,TL_diff,run,frequency_band,brain_label,power
1,1045i,right,1 Hz,17.142857,post,delta,caudalanteriorcingulate-lh,30.461920
351,1045i,right,1 Hz,17.142857,pre,delta,caudalanteriorcingulate-lh,20.076259
701,20cz9,left,1 Hz,11.428571,post,delta,caudalanteriorcingulate-lh,29.919254
1051,20cz9,left,1 Hz,11.428571,pre,delta,caudalanteriorcingulate-lh,27.156278
1401,2bbyx,left,1 Hz,26.428571,post,delta,caudalanteriorcingulate-lh,19.929565
...,...,...,...,...,...,...,...,...
9644,w1p3k,left,10 Hz,10.000000,pre,alpha,parsorbitalis-rh,14.099938
9994,z45am,right,10 Hz,28.571429,post,alpha,parsorbitalis-rh,15.734264
10344,z45am,right,10 Hz,28.571429,pre,alpha,parsorbitalis-rh,13.327674
10694,zy37y,left,20 Hz,10.714286,post,alpha,parsorbitalis-rh,9.227952


In [ ]:
df_final.to_csv("/Users/payamsadeghishabestari/codes/regTMS/data/dataframes/new_analysis_with_TL_new/df_pre_vs_post_responders.csv")

In [33]:
freq = "alpha"
df_plot = df_final.query(f'frequency_band == "{freq}"')
fig, ax = plt.subplots(1, 1, figsize=(11, 5), layout="constrained")
pal = sns.cubehelix_palette(7, rot=-.25, light=.7, as_cmap=False)
pals = [pal[1], pal[-3]]
sns.boxplot(df_plot, x="brain_label", y="power", hue="run", fill=False, hue_order=["pre", "post"], linewidth=1.4, gap=0.2, palette=pals, ax=ax)
sns.stripplot(df_plot, x="brain_label", y="power", hue="run", hue_order=["pre", "post"], dodge=True, size=5, legend=False, palette=pals, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=12))
ax.legend(frameon=False)
ax.spines[["right", "top"]].set_visible(False)
#ax.set_title(f"Significant Brain Labels @ {freq} Frequency Range")

/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_1321/629453595.py:8: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=12))


In [ ]:
## add worst protocol in the same hemi

fname = "/Users/payamsadeghishabestari/codes/regTMS/data/behavioral_data/TL_data_new.xlsx"  # changed to new
df_tl = pd.read_excel(fname)
df_tl["protcol"] = df_tl["protcol"].replace("1Hz", "1 Hz")
df_tl.rename(columns={"protcol": "protocol", "ID": "subject_ID"}, inplace=True)

df_tl['pre_avg'] = df_tl.filter(like='pre').mean(axis=1)
df_tl['post_avg'] = df_tl.filter(like='post').mean(axis=1)
df_tl.drop(columns=list(df_tl.columns[4:-2]), inplace=True)
df_tl["TL_diff"] = df_tl['post_avg'] - df_tl['pre_avg']


freq = "theta"
df_plot_sub = df_final.query(f'frequency_band == "{freq}"')

dfws_list = []
for subject in df_plot_sub["subject_ID"].unique():
    x = df_plot_sub.query(f'subject_ID == "{subject}"')
    bls = list(x["brain_label"].unique())
    hemi = x["hemisphere"].unique()

    df_tl.query(f'subject_ID == "{subject}" & hemisphere == "{hemi}"')
    min_row = df_tl.loc[df_tl['TL_diff'].idxmin()]
    worst_protocol = min_row["protocol"]

    df_w = df.query(f'frequency_band == "{freq}" & subject_ID == "{subject}" & hemisphere == "{hemi[0]}" & protocol == "{worst_protocol}" & brain_label == @bls')
    df_w["best_or_worst"] = len(df_w) * ["worst"]
    dfws_list.append(df_w)

dfws = pd.concat(dfws_list)
dfbs = df_final.query(f'frequency_band == "{freq}"')
dfbs["best_or_worst"] = len(dfbs) * ["best"]


## now caompare
for bl in dfbs["brain_label"].unique():
    df1 = dfbs.query(f'brain_label == "{bl}"')
    df2 = dfws.query(f'brain_label == "{bl}"')

    group_bs_pre = df1[df1["run"] == "pre"]["power"].values
    group_bs_post = df1[df1["run"] == "post"]["power"].values

    group_w_pre = df2[df2["run"] == "pre"]["power"].values
    group_w_post = df2[df2["run"] == "post"]["power"].values

    t_val, p_val = ttest_rel(group_bs_post - group_bs_pre, group_w_post - group_w_pre)
    if p_val < 0.05:
        print(bl, p_val)


In [81]:
bls = list(bls)
df_w.query(f'brain_label == @bls')

,subject_ID,hemisphere,protocol,run,frequency_band,brain_label,power
119852,zy37y,left,20 Hz,post,alpha,lateralorbitofrontal-lh,10.469867
119859,zy37y,left,20 Hz,post,alpha,parsorbitalis-lh,9.651796
119880,zy37y,left,20 Hz,post,alpha,frontalpole-rh,9.624557
119887,zy37y,left,20 Hz,post,alpha,lateralorbitofrontal-rh,10.495605
119894,zy37y,left,20 Hz,post,alpha,parsorbitalis-rh,9.227952
120202,zy37y,left,20 Hz,pre,alpha,lateralorbitofrontal-lh,9.292486
120209,zy37y,left,20 Hz,pre,alpha,parsorbitalis-lh,9.134350
120230,zy37y,left,20 Hz,pre,alpha,frontalpole-rh,9.316410
120237,zy37y,left,20 Hz,pre,alpha,lateralorbitofrontal-rh,9.440420
120244,zy37y,left,20 Hz,pre,alpha,parsorbitalis-rh,8.520133


In [73]:
x

,subject_ID,hemisphere,protocol,TL_diff,run,frequency_band,brain_label,power
10680,zy37y,left,20 Hz,10.714286,post,alpha,frontalpole-rh,9.624557
10652,zy37y,left,20 Hz,10.714286,post,alpha,lateralorbitofrontal-lh,10.469867
10687,zy37y,left,20 Hz,10.714286,post,alpha,lateralorbitofrontal-rh,10.495605
10659,zy37y,left,20 Hz,10.714286,post,alpha,parsorbitalis-lh,9.651796
10694,zy37y,left,20 Hz,10.714286,post,alpha,parsorbitalis-rh,9.227952


In [139]:
all_regions

['parstriangularis-lh',
 'rostralmiddlefrontal-lh',
 'parsorbitalis-lh',
 'temporalpole-rh',
 'frontalpole-lh',
 'lateralorbitofrontal-rh',
 'lateralorbitofrontal-lh',
 'rostralanteriorcingulate-rh',
 'medialorbitofrontal-rh',
 'parsopercularis-lh']

In [140]:
brain_labels = np.array(mne.read_labels_from_annot(subject="fsaverage", parc="aparc", verbose=False))
lb_names = [lb.name for lb in brain_labels]
brain_kwargs = dict(background="white", surf="pial_semi_inflated", cortex=["#b8b4ac", "#b8b4ac"])
all_regions = df_final.query(f'frequency_band == "alpha"')["brain_label"].unique()
region_idxs = np.array([lb_names.index(rg) for rg in all_regions])

df_sub = df_corr_bls.query('p_value < 0.05')
all_regions = list(df_sub["brain_label"])
region_idxs = np.array([lb_names.index(rg) for rg in all_regions])



lbs = brain_labels[region_idxs]
# colors = sns.color_palette(palette="Set1", n_colors=len(all_regions))
# colors_alpha = ["blue", "#5c8687", "#987460", "magenta", "#8c6a59"]
# colors_alpha_cor = ["green", "cyan", ""]
# colors = colors_alpha


colors_dict = {'parstriangularis-lh': "red",
                'rostralmiddlefrontal-lh': "yellow",
                'parsorbitalis-lh': "orange",
                'temporalpole-rh': "#e8b37e",
                'frontalpole-lh': "#9fa3ad",
                'lateralorbitofrontal-rh': "#8f6d5a",
                'lateralorbitofrontal-lh': "#639191",
                'rostralanteriorcingulate-rh': "#d08a84",
                'medialorbitofrontal-rh': "#d0c5cf",
                'parsopercularis-lh': "slategrey",
                'frontalpole-rh': "#a6b6b6",
                'parsorbitalis-rh': "#dabf9f"
                }

colors = [colors_dict[r] for r in all_regions]

alpha = 0.75

brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="lh", views="lateral", **brain_kwargs)
for lb, color in zip(lbs, colors):
    if lb.hemi == "lh":
        brain.add_label(lb, hemi="lh", color=color, borders=False, alpha=alpha)

brain_scr_1 = brain.screenshot()
brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="rh", views="lateral", **brain_kwargs)
for lb, color in zip(lbs, colors):
    if lb.hemi == "rh":
        brain.add_label(lb, hemi="rh", color=color, borders=False, alpha=alpha)

brain_scr_2 = brain.screenshot()

brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="lh", views="medial", **brain_kwargs)
for lb, color in zip(lbs, colors):
    if lb.hemi == "lh":
        brain.add_label(lb, hemi="lh", color=color, borders=False, alpha=alpha)

brain_scr_3 = brain.screenshot()
brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="rh", views="medial", **brain_kwargs)
for lb, color in zip(lbs, colors):
    if lb.hemi == "rh":
        brain.add_label(lb, hemi="rh", color=color, borders=False, alpha=alpha)


brain_scr_4 = brain.screenshot()
fig, axes = plt.subplots(2, 2, figsize=(9, 7))
fig.subplots_adjust(hspace=0.1)
for ax, brain in zip([axes[0][0], axes[0][1], axes[1][0], axes[1][1]], [brain_scr_1, brain_scr_2, brain_scr_3, brain_scr_4]):
    nonwhite_pix = (brain != 255).any(-1)
    nonwhite_row = nonwhite_pix.any(1)
    nonwhite_col = nonwhite_pix.any(0)
    cropped_screenshot = brain[nonwhite_row][:, nonwhite_col]
    ax.imshow(cropped_screenshot)
    ax.axis("off")


# palette_with_alpha = [mcolors.to_rgba(color, alpha=alpha) for color in colors]
# fig, ax = plt.subplots(1, 1, figsize=(16, 8))
# ax.barh(range(len(palette_with_alpha)), [1]*len(palette_with_alpha), color=palette_with_alpha, tick_label=all_regions)
# #ax.yaxis.tick_right()
# ax.set_yticklabels(all_regions, fontstyle='italic', fontsize=11, ha='left')
# ax.yaxis.set_tick_params(pad=-860)
# for spine in ax.spines.values():
#     spine.set_visible(False)

Context leak detected, msgtracer returned -1


In [ ]:
df_pivot = df_only_responders.pivot(index=["subject_ID", "protocol", "hemisphere", "TL_diff",
                                            "frequency_band", "brain_label"], columns='run', values='power')
df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
df_pivot.drop(columns=["post", "pre"], inplace=True)
df_pivot = df_pivot.reset_index()

In [ ]:
df_pivot

In [ ]:
my_dict = {"frequency_band": [], "brain_label":[], "correlation": [], "p_value":[]}

brain_labels = mne.read_labels_from_annot(subject="fsaverage", parc="aparc", verbose=False)[:-2]
lb_names = [lb.name for lb in brain_labels]
franges = ["delta", "theta", "alpha", "beta", "gamma"]
for freq in franges:
    for bl in lb_names:
        df_c = df_pivot.query(f'frequency_band == "{freq}" & brain_label == "{bl}"')
        correlation, p_value = pearsonr(df_c['TL_diff'], df_c['power_difference'])

        my_dict["frequency_band"].append(freq)
        my_dict["brain_label"].append(bl)
        my_dict["correlation"].append(correlation)
        my_dict["p_value"].append(p_value)

df_11 = pd.DataFrame(my_dict)
df_11 = df_11.sort_values(by='p_value')
# df_11.to_csv("/Users/payamsadeghishabestari/Desktop/corr_bls_responders.csv")

In [ ]:
df_11.to_csv("/Users/payamsadeghishabestari/codes/regTMS/data/dataframes/new_analysis_with_TL_new/corr_bls_responders_new.csv")

In [120]:
df_corr_bls = pd.read_csv("/Users/payamsadeghishabestari/codes/regTMS/data/dataframes/new_analysis_with_TL_new/corr_bls_responders_new.csv")


In [121]:
df_sub = df_corr_bls.query('p_value < 0.05')
all_regions = list(df_sub["brain_label"])

In [ ]:
roi = ['transversetemporal', 'bankssts', 'inferiorparietal',
        'superiorparietal', 'superiortemporal', 'supramarginal',
        'middletemporal']
roi_lh = [bl + "-lh" for bl in roi]
roi_rh = [bl + "-rh" for bl in roi]
roi = roi_lh + roi_rh

df_sub = df.query(f'brain_label == @roi')
df_sub_1 = df_sub[(df_sub['hemisphere'] == 'right') & (df_sub['brain_label'].str.endswith('rh'))]
df_sub_2 = df_sub[(df_sub['hemisphere'] == 'left') & (df_sub['brain_label'].str.endswith('lh'))]
df_sub = pd.concat([df_sub_1, df_sub_2])
df_sub = df_sub.groupby(["subject_ID", "protocol", "hemisphere", "run", "frequency_band"])['power'].sum().reset_index()
df_pivot = df_sub.pivot(index=["subject_ID", "protocol", "hemisphere", "frequency_band"], columns='run', values='power')
df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
# df_pivot.drop(columns=["post", "pre"], inplace=True)
df_pivot = df_pivot.reset_index()

df_responders_roi = pd.merge(df_best_prot, df_pivot, on=["subject_ID", "hemisphere", "protocol"])

In [ ]:
df_responders_roi

In [ ]:
franges = ["delta", "theta", "alpha", "beta", "gamma"]
ddf1 = df_responders_roi.query('protocol != "0.1 Hz" & TL_diff > 0')
ddf2 = df_responders_roi.query('protocol == "0.1 Hz"')
df_responders_roi = pd.concat([ddf1, ddf2])
for freq in franges:
    df1 = df_responders_roi.query(f'frequency_band == "{freq}" & protocol != "0.1 Hz"')
    my_subs = list(df1["subject_ID"].unique())
    df2 = df_responders_roi.query(f'frequency_band == "{freq}" & protocol == "0.1 Hz" & subject_ID == @my_subs')
    t_val, p_val = ttest_rel(df1["post"].values, df1["pre"].values)

    # print(f"pre_vs_post at {freq}: t_value = {round(t_val, 4)} and p_value= {round(p_val, 4)}")

    t_val, p_val = ttest_rel(df1["power_difference"].values, df2["power_difference"].values)
    
    # print(f"sham_vs_best_protocol at {freq}: t_value = {round(t_val, 4)} and p_value= {round(p_val, 4)}")

    corr_val = df1['TL_diff'].corr(df1['power_difference'])
    print(f"corr between TL_diff and diff_power in ROI at frange {freq}: {round(corr_val, 4)}")

In [ ]:
df2["power_difference"].values

Identify the worst protocol for the subjects in the
responder group (16 responders) == non-responder group

++ and compare the EEG power changes (post-pre)
between responder and non-responder group for ROI

In [ ]:
fname = "/Users/payamsadeghishabestari/codes/regTMS/data/behavioral_data/TL_data.xlsx"
df_tl = pd.read_excel(fname)
df_tl["protcol"] = df_tl["protcol"].replace("1Hz", "1 Hz")
df_tl.rename(columns={"protcol": "protocol", "ID": "subject_ID"}, inplace=True)
prots = ["1 Hz", "10 Hz", "20 Hz"]
hemis = ["left", "right"]

worst_protocol_dict = {"subject_ID": [], "hemisphere": [], "protocol": [], "TL_diff": []} 
for subject in df["subject_ID"].unique():
    prot_dict = {}
    for prot in prots:
        for hemi in hemis:
            df_tl_sub = df_tl.query(f'subject_ID == "{subject}" & protocol == "{prot}" & hemisphere == "{hemi}"')
            df_tl_sub['pre_avg'] = df_tl.filter(like='pre').mean(axis=1)
            df_tl_sub['post_avg'] = df_tl.filter(like='post').mean(axis=1)
            df_tl_sub["diff_TL"] = df_tl_sub['pre_avg'] - df_tl_sub['post_avg']
            prot_dict[f"{hemi}_{prot}"] = df_tl_sub["diff_TL"].values[0]
    best_protocol = max(prot_dict, key=prot_dict.get)

    min_value = min(prot_dict.values())
    worst_protocols = [key for key, value in prot_dict.items() if value == min_value]

    if isinstance(worst_protocols, list):
        for worst_protocol in worst_protocols:
            worst_protocol_dict["subject_ID"].append(subject)
            worst_protocol_dict["hemisphere"].append(worst_protocol.partition("_")[0])
            worst_protocol_dict["protocol"].append(worst_protocol.partition("_")[2])
            worst_protocol_dict["TL_diff"].append(prot_dict[worst_protocol])

            worst_protocol_dict["subject_ID"].append(subject)
            worst_protocol_dict["hemisphere"].append(worst_protocol.partition("_")[0])
            worst_protocol_dict["protocol"].append("0.1 Hz")
            worst_protocol_dict["TL_diff"].append(0)


df_worst_prot = pd.DataFrame(worst_protocol_dict)
df_only_non_responders = pd.merge(df_worst_prot, df, on=["subject_ID", "hemisphere", "protocol"])

roi = ['transversetemporal', 'bankssts', 'inferiorparietal',
        'superiorparietal', 'superiortemporal', 'supramarginal',
        'middletemporal']
roi_lh = [bl + "-lh" for bl in roi]
roi_rh = [bl + "-rh" for bl in roi]
roi = roi_lh + roi_rh

df_sub = df.query(f'brain_label == @roi')
df_sub_1 = df_sub[(df_sub['hemisphere'] == 'right') & (df_sub['brain_label'].str.endswith('rh'))]
df_sub_2 = df_sub[(df_sub['hemisphere'] == 'left') & (df_sub['brain_label'].str.endswith('lh'))]
df_sub = pd.concat([df_sub_1, df_sub_2])
df_sub = df_sub.groupby(["subject_ID", "protocol", "hemisphere", "run", "frequency_band"])['power'].sum().reset_index()
df_pivot = df_sub.pivot(index=["subject_ID", "protocol", "hemisphere", "frequency_band"], columns='run', values='power')
df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
# df_pivot.drop(columns=["post", "pre"], inplace=True)
df_pivot = df_pivot.reset_index()

df_non_responders_roi = pd.merge(df_worst_prot, df_pivot, on=["subject_ID", "hemisphere", "protocol"])
ids = list(df_responders_roi["subject_ID"].unique())
df_non_responders_roi = df_non_responders_roi.query('subject_ID in @ids & protocol != "0.1 Hz"')

filtered_x_v1 = df_non_responders_roi[df_non_responders_roi["TL_diff"] == 0]
filtered_x_v2 = df_non_responders_roi[df_non_responders_roi["TL_diff"] != 0][["subject_ID", "TL_diff", "power_difference", "frequency_band"]]
merged_df = (filtered_x_v1.groupby(["subject_ID", "frequency_band"], as_index=False).agg({"TL_diff": "first",
                                                                    "power_difference": "mean",
                                                                    "frequency_band": "first"}))
df_non_responders_roi_v2 = pd.concat([merged_df, filtered_x_v2])

In [ ]:
df_non_responders_roi_v2

In [ ]:
df_responders_roi["responder"] = ["responder"] * len(df_responders_roi)
df_non_responders_roi_v2["responder"] = ["non-responder"] * len(df_non_responders_roi_v2)
df_both = pd.concat([df_responders_roi, df_non_responders_roi_v2])

fig, ax = plt.subplots(1, 1, figsize=(8, 4))
pal = sns.cubehelix_palette(7, rot=-.25, light=.8, as_cmap=False)[3]
order = ["delta", "theta", "alpha", "beta", "gamma"]
sns.boxplot(data=df_both, x="frequency_band", y="power_difference", hue="responder", fill=False, color=pal, gap=0.2, ax=ax, order=order)
sns.stripplot(data=df_both, x="frequency_band", y="power_difference", hue="responder", dodge=True, color=pal, s=3, legend=False, ax=ax, order=order)
ax.set_ylim([-100, 250])
ax.legend(frameon=False)
ax.spines[["right", "top"]].set_visible(False)

In [ ]:
df_non_responders_roi_v2

In [ ]:
franges = ["delta", "theta", "alpha", "beta", "gamma"]
for freq in franges:
    resp_df = df_responders_roi.query(f'protocol != "0.1 Hz" & frequency_band == "{freq}"')
    non_resp_df = df_non_responders_roi_v2.query(f'frequency_band == "{freq}"')
    resp_data = resp_df["power_difference"].values
    non_resp_data = non_resp_df["power_difference"].values
    t_val, p_val = ttest_rel(resp_data, non_resp_data)
    
    if p_val < 0.05:
        print(f"at {freq} frequency range: avg power diff in responders is {round(resp_data.mean(),2)} and avg power diff in non_responders is {round(non_resp_data.mean(),2)} and significant with (t_val and p_val):({round(t_val, 4)},{round(p_val,4)})")

In [ ]:
non_resp_df

Picture 3.c (anatomical label)

In [ ]:
## anatomical ROI
roi = ['transversetemporal', 'bankssts', 'inferiorparietal',
        'superiorparietal', 'superiortemporal', 'supramarginal',
        'middletemporal']
brain_labels = mne.read_labels_from_annot(subject="fsaverage", parc="aparc", verbose=False)
lb_names = [lb.name for lb in brain_labels]
roi_rh = brain_labels[-2]
roi_lh = brain_labels[-3]
roi_rh_list = [roi_rh]
roi_lh_list = [roi_lh]
for item in roi[1:]:
    idx_rh = lb_names.index(item + "-rh")
    idx_lh = lb_names.index(item + "-lh")
    roi_rh += brain_labels[idx_rh]
    roi_lh += brain_labels[idx_lh]
    roi_rh_list.append(brain_labels[idx_rh])
    roi_lh_list.append(brain_labels[idx_lh])

brain_kwargs = dict(background="white", surf="pial_semi_inflated", cortex=["#b8b4ac", "#b8b4ac"])
alpha = 0.6
color = sns.cubehelix_palette(7, rot=-.25, light=.7, as_cmap=False)[6]

brain_scrs = []
for view in ["lateral", "medial"]:
    for hemi, sub_roi, sub_roi_list in zip(["lh", "rh"], [roi_lh, roi_rh], [roi_lh_list, roi_rh_list]):
        brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi=hemi, views=view, **brain_kwargs)
        brain.add_label(sub_roi, hemi=hemi, color=color, borders=False, alpha=0.5)
        for item in sub_roi_list:
            brain.add_label(item, hemi=hemi, color=color, borders=True, alpha=0.6)
        brain_scrs.append(brain.screenshot())

fig, axes = plt.subplots(2, 2, figsize=(9, 7))
fig.subplots_adjust(hspace=0.1)
for ax, brain in zip([axes[0][0], axes[0][1], axes[1][0], axes[1][1]], brain_scrs):
    nonwhite_pix = (brain != 255).any(-1)
    nonwhite_row = nonwhite_pix.any(1)
    nonwhite_col = nonwhite_pix.any(0)
    cropped_screenshot = brain[nonwhite_row][:, nonwhite_col]
    ax.imshow(cropped_screenshot)
    ax.axis("off")

fig.savefig(Path.cwd().parent / "figures" / "to_stefan" / f"defined_ROI.pdf")

Picture 4.a (responders)

In [ ]:
## defining best protocol per subject
fname = "/Users/payamsadeghishabestari/codes/regTMS/data/behavioral_data/TL_data.xlsx"
df_tl = pd.read_excel(fname)
df_tl["protcol"] = df_tl["protcol"].replace("1Hz", "1 Hz")
df_tl.rename(columns={"protcol": "protocol", "ID": "subject_ID"}, inplace=True)
prots = ["1 Hz", "10 Hz", "20 Hz"]
hemis = ["left", "right"]

best_protocol_dict = {"subject_ID": [], "hemisphere": [], "protocol": [], "TL_diff": []} 
for subject in df["subject_ID"].unique():
    prot_dict = {}
    for prot in prots:
        for hemi in hemis:
            df_tl_sub = df_tl.query(f'subject_ID == "{subject}" & protocol == "{prot}" & hemisphere == "{hemi}"')
            df_tl_sub['pre_avg'] = df_tl.filter(like='pre').mean(axis=1)
            df_tl_sub['post_avg'] = df_tl.filter(like='post').mean(axis=1)
            df_tl_sub["diff_TL"] = df_tl_sub['pre_avg'] - df_tl_sub['post_avg']
            prot_dict[f"{hemi}_{prot}"] = df_tl_sub["diff_TL"].values[0]
    best_protocol = max(prot_dict, key=prot_dict.get)
    
    best_protocol_dict["subject_ID"].append(subject)
    best_protocol_dict["hemisphere"].append(best_protocol.partition("_")[0])
    best_protocol_dict["protocol"].append(best_protocol.partition("_")[2])
    best_protocol_dict["TL_diff"].append(prot_dict[best_protocol])

    best_protocol_dict["subject_ID"].append(subject)
    best_protocol_dict["hemisphere"].append(best_protocol.partition("_")[0])
    best_protocol_dict["protocol"].append("0.1 Hz")
    best_protocol_dict["TL_diff"].append(0)

df_best_prot = pd.DataFrame(best_protocol_dict)
df_best_prot = df_best_prot.query('TL_diff >= 0')
df_only_responders = pd.merge(df_best_prot, df, on=["subject_ID", "hemisphere", "protocol"])

## pre vs post comparison
df_sub = df_only_responders.query('protocol != "0.1 Hz"')
brain_labels = mne.read_labels_from_annot(subject="fsaverage", parc="aparc", verbose=False)
lb_names = [lb.name for lb in brain_labels]
franges = ["delta", "theta", "alpha", "beta", "gamma"]

responders_pre_vs_post = {"frange": [], "brain_label": [], "post - pre": [], "t_val": [], "p_val": []}
for frange in franges:
    for bl in lb_names:
        df_sub_sub = df_sub.query(f'brain_label == "{bl}" & frequency_band == "{frange}"')
        group_pre = df_sub_sub[df_sub_sub["run"] == "pre"]["power"].values
        group_post = df_sub_sub[df_sub_sub["run"] == "post"]["power"].values
        t_val, p_val = ttest_rel(group_pre, group_post)
        
        if p_val < 0.05:
            responders_pre_vs_post["frange"].append(frange)
            responders_pre_vs_post["brain_label"].append(bl)
            responders_pre_vs_post["post - pre"].append(group_post.mean() - group_pre.mean())
            responders_pre_vs_post["t_val"].append(t_val)
            responders_pre_vs_post["p_val"].append(p_val)

df_responders_pre_vs_post = pd.DataFrame(responders_pre_vs_post)

## comparing vs sham
df_only_responders.drop(columns="hemisphere", inplace=True)
df_pivot = df_only_responders.pivot(index=["subject_ID", "protocol", 'frequency_band', 'brain_label'], columns='run', values='power')
df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
df_pivot.drop(columns=['post', 'pre'], inplace=True)
df_pivot.reset_index(inplace=True)

responders_sham_vs_varum = {"frange": [], "brain_label": [], "varum - sham": [], "t_val": [], "p_val": []}
for frange in franges:
    for bl in lb_names:
        df_sub_sub = df_pivot.query(f'brain_label == "{bl}" & frequency_band == "{frange}"')
        group_sham = df_sub_sub[df_sub_sub["protocol"] == "0.1 Hz"]["power_difference"].values
        group_varum = df_sub_sub[df_sub_sub["protocol"] != "0.1 Hz"]["power_difference"].values
        
        assert group_sham.shape == group_varum.shape, "sth wrong"
        t_val, p_val = ttest_rel(group_sham, group_varum)

        if p_val < 0.05:
            responders_sham_vs_varum["frange"].append(frange)
            responders_sham_vs_varum["brain_label"].append(bl)
            responders_sham_vs_varum["varum - sham"].append(group_varum.mean() - group_sham.mean())
            responders_sham_vs_varum["t_val"].append(t_val)
            responders_sham_vs_varum["p_val"].append(p_val)

df_responders_sham_vs_varum = pd.DataFrame(responders_sham_vs_varum)

In [ ]:
df_only_responders

Picture 5.a

In [ ]:
abb_names = {
    'left_20 Hz_gamma': ['FG-rh'],
    'right_10 Hz_alpha': ['EC-lh', 'PrCu-lh', 'LG-lh', 'LO-lh', 'Cu-lh', 'PC-lh'],
    'right_10 Hz_gamma': ['TP-rh', 'SMG-rh', 'PoCG-rh'],
    'right_20 Hz_delta': ['POP-rh', 'RAC-lh', 'RMF-rh', 'PT-rh', 'Ins-rh', 'PO-rh', 'STG-rh', 'TP-lh', 'EC-lh', 'FP-rh', 'MOF-rh', 'MOF-lh', 'FP-lh', 'EC-rh', 'SFG-rh', 'LOF-lh', 'LOF-rh', 'RAC-rh', 'CMF-rh', 'TP-rh'],
    'right_20 Hz_theta': ['POP-rh', 'Ins-rh', 'PO-rh', 'LOF-rh', 'FP-rh', 'RAC-lh', 'RAC-rh', 'CMF-rh', 'MOF-lh', 'FP-lh', 'MOF-rh', 'EC-rh', 'SFG-rh', 'TP-rh', 'RMF-rh', 'PT-rh'],
    'right_20 Hz_alpha': ['ITG-rh', 'PCG-rh', 'CMF-rh', 'SFG-rh', 'EC-rh', 'FG-rh', 'CAC-rh'],
    'right_20 Hz_gamma': ['ITG-rh', 'Ins-rh', 'LOF-rh', 'MOF-rh', 'EC-rh', 'MTG-rh']
}
all_bl = []
for item in comm_dict.values():
    all_bl += item

all_bl_abv = []
for item in abb_names.values():
    all_bl_abv += item

abb_names_dict = dict(zip(all_bl, all_bl_abv))

protocols = ["10 Hz", "20 Hz"]
franges = ["delta", "theta", "alpha", "beta", "gamma"]
hemis = ["left", "right"]
combs = product(hemis, protocols, franges)

size = 11
pal = sns.cubehelix_palette(7, rot=-.25, light=.7, as_cmap=False)
pals = [pal[1], pal[-3]]

for hemi, protocol, frange in combs:
    for key, value in comm_dict.items():
        abb_vals = abb_names[key]
        if hemi in key and protocol in key and frange in key:
            df_sub = df.query(f'hemisphere == "{hemi}" & protocol == "{protocol}" & frequency_band == "{frange}" & brain_label == @value')
            if frange == "delta": 
                y_size = 11; ylim = 400
            if frange == "theta":
                y_size = 11; ylim = 100
            if frange == "alpha":
                if protocol == "10 Hz": y_size = 4.6; ylim = None
                if protocol == "20 Hz": y_size = 5.6; ylim = None
            if frange == "gamma" and hemi == "left": y_size = 2; ylim = None
            if frange == "gamma" and hemi == "right":
                if protocol == "10 Hz": y_size = 3.6; ylim = None
                if protocol == "20 Hz": y_size = 4.9; ylim = None

            my_bl_dict = {}
            for bl in value:
                data_pre = df_sub.query(f'brain_label == "{bl}" & run == "pre"')["power"].values
                data_post = df_sub.query(f'brain_label == "{bl}" & run == "post"')["power"].values
                t_val, p_val = ttest_rel(data_pre, data_post)
                my_bl_dict[bl] = p_val
                sorted_dict = dict(sorted(my_bl_dict.items(), key=lambda item: item[1]))
            order = list(sorted_dict.keys())
            
            fig, ax = plt.subplots(1, 1, figsize=(y_size, 5), layout="constrained")
            sns.boxplot(df_sub, x="brain_label", y="power", hue="run", fill=False, hue_order=["pre", "post"], linewidth=1.4, order=order, palette=pals, ax=ax)
            sns.stripplot(df_sub, x="brain_label", y="power", hue="run", hue_order=["pre", "post"], dodge=True, size=3, legend=False, order=order, palette=pals, ax=ax)

            abb_vals = [abb_names_dict[item] for item in order]
            ax.set_xticklabels(abb_vals, fontdict=dict(size=size))
            ax.legend(frameon=False)
            ax.spines[["right", "top"]].set_visible(False)
            ax.set_title(f"{hemi}_{protocol}_{frange}")
            ax.set_ylim([None, ylim])

            fig.savefig(Path.cwd().parent / "figures" / "to_stefan" / "supp" / f"whole_{key}.pdf")

Boxplots all subjects (pre vs post)

In [ ]:
dir = Path.cwd().parent / "data" / "dataframes" / "results"
pal = sns.cubehelix_palette(7, rot=-.25, light=.7, as_cmap=False)
pals = [pal[1], pal[-2]]

fname = dir / "whole_bls.json"
with open(fname, "r") as f:
    results_whole = json.load(f)

fname = dir / "responders_bls.json"
with open(fname, "r") as f:
    results_responders = json.load(f)

bls_dict_whole = {"delta": [], "theta": [], "gamma": []}
for item in results_whole:
    if len(re.findall(r'\d+', item)[-1]) != 4:
        item = f"{item}0"

    if "delta" in item:
        bls_dict_whole["delta"].append(item[18:-22])
    if "theta" in item:
        bls_dict_whole["theta"].append(item[18:-22])
    if "gamma" in item:
        bls_dict_whole["gamma"].append(item[18:-22])

bls_dict_responders = {"delta": [], "theta": [], "gamma": []}
for item in results_responders:
    if len(re.findall(r'\d+', item)[-1]) != 4:
        item = f"{item}0"
    
    if "delta" in item:
        bls_dict_responders["delta"].append(item[18:-22])
    if "theta" in item:
        bls_dict_responders["theta"].append(item[18:-22])
    if "gamma" in item:
        bls_dict_responders["gamma"].append(item[18:-22])

In [ ]:
comm_dict.keys()

In [ ]:
pal = sns.cubehelix_palette(7, rot=-.25, light=.7, as_cmap=False)
pals = [pal[1], pal[-2]]
size = 10
protocols = ["0.1 Hz", "1 Hz", "10 Hz", "20 Hz"]
franges = ["delta", "theta", "alpha", "beta", "gamma"]
hemis = ["left", "right"]
combs = product(hemis, protocols, franges)
for hemi, protocol, frange in combs:
    for key, value in comm_dict.items():
        if hemi in key and protocol in key and frange in key:
            df_sub = df.query(f'hemisphere == "{hemi}" & protocol == "{protocol}" & frequency_band == "{frange}" & brain_label == @value')
            fig, ax = plt.subplots(1, 1, figsize=(11, 5), layout="constrained")
            sns.boxplot(df_sub, x="brain_label", y="power", hue="run", fill=False, hue_order=["pre", "post"], linewidth=1.4, palette=pals, ax=ax)
            sns.stripplot(df_sub, x="brain_label", y="power", hue="run", hue_order=["pre", "post"], dodge=True, size=3, legend=False, palette=pals, ax=ax)
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=size))
            ax.legend(frameon=False)
            ax.spines[["right", "top"]].set_visible(False)
            ax.set_title(f"hemi: {hemi} & protocol: {protocol} & frange: {frange}")
            fig.savefig(f"/Users/payamsadeghishabestari/codes/regTMS/figures_new/whole_{hemi}_{protocol}_{frange}.pdf")

In [ ]:
## whole subjects
size = 10
frange = "delta"
lbs = bls_dict_whole[frange]
df_sub = df.query(f'hemisphere == "right" & protocol == "20 Hz" & frequency_band == "{frange}" & brain_label == @lbs')
fig, ax = plt.subplots(1, 1, figsize=(11, 5), layout="constrained")
sns.boxplot(df_sub, x="brain_label", y="power", hue="run", fill=False, hue_order=["pre", "post"], linewidth=1.4, palette=pals, ax=ax)
sns.stripplot(df_sub, x="brain_label", y="power", hue="run", hue_order=["pre", "post"], dodge=True, size=3, legend=False, palette=pals, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=size))
ax.legend(frameon=False)
ax.spines[["right", "top"]].set_visible(False)
ax.set_title(f"Significant Brain Labels @ {frange} Frequency Range")
fig.savefig(Path.cwd().parent / "figures" / f"whole_{frange}.pdf")

frange = "theta"
lbs = bls_dict_whole[frange]
df_sub = df.query(f'hemisphere == "right" & protocol == "20 Hz" & frequency_band == "{frange}" & brain_label == @lbs')
fig, ax = plt.subplots(1, 1, figsize=(11, 5), layout="constrained")
sns.boxplot(df_sub, x="brain_label", y="power", hue="run", fill=False, hue_order=["pre", "post"], linewidth=1.4, palette=pals, ax=ax)
sns.stripplot(df_sub, x="brain_label", y="power", hue="run", hue_order=["pre", "post"], dodge=True, size=3, legend=False, palette=pals, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=size))
ax.legend(frameon=False)
ax.spines[["right", "top"]].set_visible(False)
ax.set_title(f"Significant Brain Labels @ {frange} Frequency Range")
fig.savefig(Path.cwd().parent / "figures" / f"whole_{frange}.pdf")

frange = "gamma"
lbs = bls_dict_whole[frange]
df_sub = df.query(f'hemisphere == "right" & protocol == "20 Hz" & frequency_band == "{frange}" & brain_label == @lbs')
fig, ax = plt.subplots(1, 1, figsize=(5, 5), layout="constrained")
sns.boxplot(df_sub, x="brain_label", y="power", hue="run", fill=False, hue_order=["pre", "post"], linewidth=1.7, palette=pals, gap=0.2, ax=ax)
sns.stripplot(df_sub, x="brain_label", y="power", hue="run", hue_order=["pre", "post"], dodge=True, size=5, legend=False, palette=pals, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=size))
ax.legend(frameon=False)
ax.spines[["right", "top"]].set_visible(False)
ax.set_title(f"Significant Brain Labels @ {frange} Frequency Range")
fig.savefig(Path.cwd().parent / "figures" / f"whole_{frange}.pdf")

In [ ]:
## define the order of xlabels wrt p_vals
def sort_brain_labels(df_sub):
    t_vals = {}
    p_vals = {}
    for bl in df_sub['brain_label'].unique():
        data_pre = df_sub.query(f'run == "pre" & brain_label == "{bl}"')["power"].to_numpy()
        data_post = df_sub.query(f'run == "post" & brain_label == "{bl}"')["power"].to_numpy()
        t_val, p_val = ttest_ind(data_pre, data_post)
        t_vals[bl] = t_val
        p_vals[bl] = p_val

    sorted_p_vals = dict(sorted(p_vals.items(), key=lambda item: item[1]))
    order = list(sorted_p_vals.keys())
    return order

In [ ]:
## only responders
non_responders = subjects_to_drop_dict["right_20 Hz"]
df_1 = df.query('subject_ID != @non_responders')
size = 10

frange = "delta"
lbs = bls_dict_responders[frange]
df_sub = df_1.query(f'hemisphere == "right" & protocol == "20 Hz" & frequency_band == "{frange}" & brain_label == @lbs')
order_delta = sort_brain_labels(df_sub)
fig, ax = plt.subplots(1, 1, figsize=(6, 4), layout="constrained")
sns.boxplot(df_sub, x="brain_label", y="power", hue="run", fill=False, hue_order=["pre", "post"], linewidth=1.4, palette=pals, gap=0.2, order=order_delta, ax=ax)
sns.stripplot(df_sub, x="brain_label", y="power", hue="run", hue_order=["pre", "post"], dodge=True, size=5, legend=False, palette=pals, order=order_delta, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=size))
ax.legend(frameon=False)
ax.spines[["right", "top"]].set_visible(False)
ax.set_title(f"Significant Brain Labels @ {frange} Frequency Range")
#fig.savefig(Path.cwd().parent / "figures" / f"responders_{frange}.pdf")

frange = "theta"
lbs = bls_dict_responders[frange]
df_sub = df_1.query(f'hemisphere == "right" & protocol == "20 Hz" & frequency_band == "{frange}" & brain_label == @lbs')
order_theta = sort_brain_labels(df_sub)
fig, ax = plt.subplots(1, 1, figsize=(11, 5), layout="constrained")
sns.boxplot(df_sub, x="brain_label", y="power", hue="run", fill=False, hue_order=["pre", "post"], linewidth=1.4, palette=pals, order=order_theta, ax=ax)
sns.stripplot(df_sub, x="brain_label", y="power", hue="run", hue_order=["pre", "post"], dodge=True, size=4, legend=False, palette=pals, order=order_theta, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=size))
ax.legend(frameon=False)
ax.spines[["right", "top"]].set_visible(False)
ax.set_title(f"Significant Brain Labels @ {frange} Frequency Range")
#fig.savefig(Path.cwd().parent / "figures" / f"responders_{frange}.pdf")

frange = "gamma"
lbs = bls_dict_responders[frange]
df_sub = df_1.query(f'hemisphere == "right" & protocol == "20 Hz" & frequency_band == "{frange}" & brain_label == @lbs')
order_gamma = sort_brain_labels(df_sub)
fig, ax = plt.subplots(1, 1, figsize=(11, 5), layout="constrained")
sns.boxplot(df_sub, x="brain_label", y="power", hue="run", fill=False, hue_order=["pre", "post"], linewidth=1.4, palette=pals, order=order_gamma, ax=ax)
sns.stripplot(df_sub, x="brain_label", y="power", hue="run", hue_order=["pre", "post"], dodge=True, size=4, legend=False, palette=pals, order=order_gamma, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=size))
ax.legend(frameon=False)
ax.spines[["right", "top"]].set_visible(False)
ax.set_title(f"Significant Brain Labels @ {frange} Frequency Range")
#fig.savefig(Path.cwd().parent / "figures" / f"responders_{frange}.pdf")

Boxplots all subjects (20 Hz vs Sham)

In [ ]:
fname = Path.cwd().parent / "data" / "dataframes" / "results" / "20_Hz_vs_sham_whole.json"
with open(fname, "r") as f:
    results_whole = json.load(f)

fname = Path.cwd().parent / "data" / "dataframes" / "results" / "20_Hz_vs_sham_responders.json"
with open(fname, "r") as f:
    results_responders = json.load(f)

bls_dict_responders = {"delta": [], "theta": [], "beta": [], "gamma": []}
for item in results_responders:
    if len(re.findall(r'\d+', item)[-1]) != 4:
        item = f"{item}0"
    if "delta" in item:
        bls_dict_responders["delta"].append(item[18:-22])
    if "theta" in item:
        bls_dict_responders["theta"].append(item[18:-22])
    if "beta" in item:
        bls_dict_responders["beta"].append(item[17:-22])    
    if "gamma" in item:
        bls_dict_responders["gamma"].append(item[18:-22])



In [ ]:
non_responders = subjects_to_drop_dict["right_20 Hz"]
df_1 = df.query('subject_ID != @non_responders')

frange = "delta"
prots = ["0.1 Hz", "20 Hz"]
lbs = bls_dict_responders[frange]
df_sub = df_1.query(f'hemisphere == "right" & protocol == @prots & frequency_band == "{frange}" & brain_label == @lbs')
df_pivot = df_sub.pivot(index=['subject_ID', 'brain_label', 'protocol'], columns='run', values='power')
df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
df_2 = df_pivot.drop(columns=['post', 'pre'])

fig, ax = plt.subplots(1, 1, figsize=(6, 5), layout="constrained")
sns.boxplot(df_2, x="brain_label", y="power_difference", hue="protocol", fill=False, hue_order=["0.1 Hz", "20 Hz"], linewidth=1.4, gap=0.2, order=order_delta, palette=pals, ax=ax)
sns.stripplot(df_2, x="brain_label", y="power_difference", hue="protocol", hue_order=["0.1 Hz", "20 Hz"], dodge=True, size=5, legend=False, order=order_delta, palette=pals, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=12))
ax.legend(frameon=False)
ax.spines[["right", "top"]].set_visible(False)
ax.set_title(f"Significant Brain Labels @ {frange} Frequency Range")
fig.savefig(Path.cwd().parent / "figures" / f"responders_vs_sham_{frange}.pdf")

########
frange = "theta"
prots = ["0.1 Hz", "20 Hz"]
lbs = bls_dict_responders[frange]
df_sub = df_1.query(f'hemisphere == "right" & protocol == @prots & frequency_band == "{frange}" & brain_label == @lbs')
df_pivot = df_sub.pivot(index=['subject_ID', 'brain_label', 'protocol'], columns='run', values='power')
df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
df_2 = df_pivot.drop(columns=['post', 'pre'])

fig, ax = plt.subplots(1, 1, figsize=(11, 5), layout="constrained")
sns.boxplot(df_2, x="brain_label", y="power_difference", hue="protocol", fill=False, hue_order=["0.1 Hz", "20 Hz"], linewidth=1.4, order=order_theta, palette=pals, ax=ax)
sns.stripplot(df_2, x="brain_label", y="power_difference", hue="protocol", hue_order=["0.1 Hz", "20 Hz"], dodge=True, size=4, legend=False, order=order_theta, palette=pals, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=12))
ax.legend(frameon=False)
ax.spines[["right", "top"]].set_visible(False)
ax.set_title(f"Significant Brain Labels @ {frange} Frequency Range")
ax.set_ylim([-20, 60])
fig.savefig(Path.cwd().parent / "figures" / f"responders_vs_sham_{frange}.pdf")

########
frange = "beta"
prots = ["0.1 Hz", "20 Hz"]
lbs = bls_dict_responders[frange]
df_sub = df_1.query(f'hemisphere == "right" & protocol == @prots & frequency_band == "{frange}" & brain_label == @lbs')
df_pivot = df_sub.pivot(index=['subject_ID', 'brain_label', 'protocol'], columns='run', values='power')
df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
df_2 = df_pivot.drop(columns=['post', 'pre'])

fig, ax = plt.subplots(1, 1, figsize=(11, 5), layout="constrained")
sns.boxplot(df_2, x="brain_label", y="power_difference", hue="protocol", fill=False, hue_order=["0.1 Hz", "20 Hz"], linewidth=1.4, palette=pals, ax=ax)
sns.stripplot(df_2, x="brain_label", y="power_difference", hue="protocol", hue_order=["0.1 Hz", "20 Hz"], dodge=True, size=4, legend=False, palette=pals, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=12))
ax.legend(frameon=False)
ax.spines[["right", "top"]].set_visible(False)
ax.set_title(f"Significant Brain Labels @ {frange} Frequency Range")
fig.savefig(Path.cwd().parent / "figures" / f"responders_vs_sham_{frange}.pdf")

########
frange = "gamma"
prots = ["0.1 Hz", "20 Hz"]
lbs = bls_dict_responders[frange]
df_sub = df_1.query(f'hemisphere == "right" & protocol == @prots & frequency_band == "{frange}" & brain_label == @lbs')
df_pivot = df_sub.pivot(index=['subject_ID', 'brain_label', 'protocol'], columns='run', values='power')
df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
df_2 = df_pivot.drop(columns=['post', 'pre'])

fig, ax = plt.subplots(1, 1, figsize=(11, 5), layout="constrained")
sns.boxplot(df_2, x="brain_label", y="power_difference", hue="protocol", fill=False, hue_order=["0.1 Hz", "20 Hz"], linewidth=1.4, order=order_gamma, palette=pals, ax=ax)
sns.stripplot(df_2, x="brain_label", y="power_difference", hue="protocol", hue_order=["0.1 Hz", "20 Hz"], dodge=True, size=4, legend=False, order=order_gamma, palette=pals, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=12))
ax.legend(frameon=False)
ax.spines[["right", "top"]].set_visible(False)
ax.set_title(f"Significant Brain Labels @ {frange} Frequency Range")
fig.savefig(Path.cwd().parent / "figures" / f"responders_vs_sham_{frange}.pdf")

for whole subjects

In [ ]:
df_1 = df

frange = "delta"
prots = ["0.1 Hz", "20 Hz"]
lbs = bls_dict_whole[frange]
df_sub = df_1.query(f'hemisphere == "right" & protocol == @prots & frequency_band == "{frange}" & brain_label == @lbs')
df_pivot = df_sub.pivot(index=['subject_ID', 'brain_label', 'protocol'], columns='run', values='power')
df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
df_2 = df_pivot.drop(columns=['post', 'pre'])
order_delta = sort_brain_labels(df_sub)
fig, ax = plt.subplots(1, 1, figsize=(11, 5), layout="constrained")
sns.boxplot(df_2, x="brain_label", y="power_difference", hue="protocol", fill=False, hue_order=["0.1 Hz", "20 Hz"], linewidth=1.4, gap=0.2, palette=pals, ax=ax)
sns.stripplot(df_2, x="brain_label", y="power_difference", hue="protocol", hue_order=["0.1 Hz", "20 Hz"], dodge=True, size=5, legend=False, palette=pals, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=12))
ax.legend(frameon=False)
ax.spines[["right", "top"]].set_visible(False)
ax.set_title(f"Significant Brain Labels @ {frange} Frequency Range")
fig.savefig(Path.cwd().parent / "figures" / f"whole_vs_sham_{frange}.pdf")

########
frange = "theta"
prots = ["0.1 Hz", "20 Hz"]
lbs = bls_dict_whole[frange]
df_sub = df_1.query(f'hemisphere == "right" & protocol == @prots & frequency_band == "{frange}" & brain_label == @lbs')
df_pivot = df_sub.pivot(index=['subject_ID', 'brain_label', 'protocol'], columns='run', values='power')
df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
df_2 = df_pivot.drop(columns=['post', 'pre'])
order_theta = sort_brain_labels(df_sub)
fig, ax = plt.subplots(1, 1, figsize=(11, 5), layout="constrained")
sns.boxplot(df_2, x="brain_label", y="power_difference", hue="protocol", fill=False, hue_order=["0.1 Hz", "20 Hz"], linewidth=1.4, palette=pals, ax=ax)
sns.stripplot(df_2, x="brain_label", y="power_difference", hue="protocol", hue_order=["0.1 Hz", "20 Hz"], dodge=True, size=4, legend=False, palette=pals, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=12))
ax.legend(frameon=False)
ax.spines[["right", "top"]].set_visible(False)
ax.set_title(f"Significant Brain Labels @ {frange} Frequency Range")
ax.set_ylim([-20, 60])
fig.savefig(Path.cwd().parent / "figures" / f"whole_vs_sham_{frange}.pdf")

########
# frange = "beta"
# prots = ["0.1 Hz", "20 Hz"]
# lbs = bls_dict_whole[frange]
# df_sub = df_1.query(f'hemisphere == "right" & protocol == @prots & frequency_band == "{frange}" & brain_label == @lbs')
# df_pivot = df_sub.pivot(index=['subject_ID', 'brain_label', 'protocol'], columns='run', values='power')
# df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
# df_2 = df_pivot.drop(columns=['post', 'pre'])

# fig, ax = plt.subplots(1, 1, figsize=(11, 5), layout="constrained")
# sns.boxplot(df_2, x="brain_label", y="power_difference", hue="protocol", fill=False, hue_order=["0.1 Hz", "20 Hz"], linewidth=1.4, palette=pals, ax=ax)
# sns.stripplot(df_2, x="brain_label", y="power_difference", hue="protocol", hue_order=["0.1 Hz", "20 Hz"], dodge=True, size=4, legend=False, palette=pals, ax=ax)
# ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=12))
# ax.legend(frameon=False)
# ax.spines[["right", "top"]].set_visible(False)
# ax.set_title(f"Significant Brain Labels @ {frange} Frequency Range")
#fig.savefig(Path.cwd().parent / "figures" / f"whole_vs_sham_{frange}.pdf")

########
frange = "gamma"
prots = ["0.1 Hz", "20 Hz"]
lbs = bls_dict_whole[frange]
df_sub = df_1.query(f'hemisphere == "right" & protocol == @prots & frequency_band == "{frange}" & brain_label == @lbs')
df_pivot = df_sub.pivot(index=['subject_ID', 'brain_label', 'protocol'], columns='run', values='power')
df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
df_2 = df_pivot.drop(columns=['post', 'pre'])

fig, ax = plt.subplots(1, 1, figsize=(2, 5), layout="constrained")
sns.boxplot(df_2, x="brain_label", y="power_difference", hue="protocol", fill=False, hue_order=["0.1 Hz", "20 Hz"], linewidth=1.4, palette=pals, ax=ax)
sns.stripplot(df_2, x="brain_label", y="power_difference", hue="protocol", hue_order=["0.1 Hz", "20 Hz"], dodge=True, size=4, legend=False, palette=pals, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=12))
ax.legend(frameon=False)
ax.spines[["right", "top"]].set_visible(False)
ax.set_title(f"Significant Brain Labels @ {frange} Frequency Range")
fig.savefig(Path.cwd().parent / "figures" / f"whole_vs_sham_{frange}.pdf")

responders vs non-responders

In [ ]:
non_responders = subjects_to_drop_dict["right_20 Hz"]
df['response_status'] = df['subject_ID'].apply(lambda x: 'non-responder' if x in non_responders else 'responder')
size = 10

frange = "delta"
lbs = bls_dict_responders[frange]
df_sub = df.query(f'hemisphere == "right" & protocol == "20 Hz" & frequency_band == "{frange}" & brain_label == @lbs')
df_pivot = df_sub.pivot(index=['subject_ID', 'brain_label', 'response_status'], columns='run', values='power')
df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
df_2 = df_pivot.drop(columns=['post', 'pre'])
fig, ax = plt.subplots(1, 1, figsize=(6, 5), layout="constrained")
sns.boxplot(df_2, x="brain_label", y="power_difference", hue="response_status", fill=False, hue_order=["non-responder", "responder"], linewidth=1.4, gap=0.2, order=order_delta, palette=pals, ax=ax)
sns.stripplot(df_2, x="brain_label", y="power_difference", hue="response_status", hue_order=["non-responder", "responder"], dodge=True, size=5, legend=False, order=order_delta, palette=pals, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=12))
ax.legend(frameon=False)
ax.spines[["right", "top"]].set_visible(False)
ax.set_title(f"Responders vs non-responders @ {frange} Frequency Range")
fig.savefig(Path.cwd().parent / "figures" / f"responder_vs_non_{frange}.pdf")

frange = "theta"
lbs = bls_dict_responders[frange]
df_sub = df.query(f'hemisphere == "right" & protocol == "20 Hz" & frequency_band == "{frange}" & brain_label == @lbs')
df_pivot = df_sub.pivot(index=['subject_ID', 'brain_label', 'response_status'], columns='run', values='power')
df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
df_2 = df_pivot.drop(columns=['post', 'pre'])
fig, ax = plt.subplots(1, 1, figsize=(11, 5), layout="constrained")
sns.boxplot(df_2, x="brain_label", y="power_difference", hue="response_status", fill=False, hue_order=["non-responder", "responder"], linewidth=1.4, gap=0.2, order=order_theta, palette=pals, ax=ax)
sns.stripplot(df_2, x="brain_label", y="power_difference", hue="response_status", hue_order=["non-responder", "responder"], dodge=True, size=5, legend=False, order=order_theta, palette=pals, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=12))
ax.legend(frameon=False)
ax.spines[["right", "top"]].set_visible(False)
ax.set_title(f"Responders vs non-responders @ {frange} Frequency Range")
fig.savefig(Path.cwd().parent / "figures" / f"responder_vs_non_{frange}.pdf")

frange = "gamma"
lbs = bls_dict_responders[frange]
df_sub = df.query(f'hemisphere == "right" & protocol == "20 Hz" & frequency_band == "{frange}" & brain_label == @lbs')
df_pivot = df_sub.pivot(index=['subject_ID', 'brain_label', 'response_status'], columns='run', values='power')
df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
df_2 = df_pivot.drop(columns=['post', 'pre'])
fig, ax = plt.subplots(1, 1, figsize=(11, 5), layout="constrained")
sns.boxplot(df_2, x="brain_label", y="power_difference", hue="response_status", fill=False, hue_order=["non-responder", "responder"], linewidth=1.4, gap=0.2, order=order_gamma, palette=pals, ax=ax)
sns.stripplot(df_2, x="brain_label", y="power_difference", hue="response_status", hue_order=["non-responder", "responder"], dodge=True, size=5, legend=False, order=order_gamma, palette=pals, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict=dict(size=12))
ax.legend(frameon=False)
ax.spines[["right", "top"]].set_visible(False)
ax.set_title(f"Responders vs non-responders @ {frange} Frequency Range")
fig.savefig(Path.cwd().parent / "figures" / f"responder_vs_non_{frange}.pdf")
